# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [1]:
import redis

In [2]:
r = redis.Redis(host='localhost', port=6379)

In [3]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [4]:
r.set('greetings', 'hello world')
res = r.get('greetings')
print(res)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [5]:
r.set('connections', '0')

True

In [6]:
new_value=r.incr('connections', 2)
print(new_value)

2


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [7]:
colors = ['red','black','blue']
r.rpush('Maliste',*colors)

3

**Q** : Display the second element of the list with `LINDEX`

In [8]:
value2=r.lindex('Maliste',1)
print(value2)

b'black'


**Q** : Display the full list with `LRANGE`

In [9]:
values = r.lrange('Maliste',0,2)
print(values)

[b'red', b'black', b'blue']


### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [10]:
r.sadd('nosql', *{'redis','mongodb','orientdb'})

3

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [11]:
r.sismember('nosql','mysql')

0

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [12]:
r.sadd('nosql','hbase')
elements=r.smembers('nosql')
print(elements)

{b'orientdb', b'mongodb', b'redis', b'hbase'}


### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [13]:
city_scores = {
    'Agen': 10, 'Bordeaux': 33, 'Brive': 32, 'Castres': 29,
    'Clermont': 38, 'Grenoble': 24, 'La Rochelle': 26,
    'Montpellier': 32, 'Oyonnax': 14, 'Pau': 20,
    'Racing': 40, 'Stade Français': 22, 'Toulon': 36, 'Toulouse': 36
}

In [14]:
r.zadd('city_scores',city_scores)
r.zrange('city_scores', 0, -1, withscores=True)

[(b'Agen', 10.0),
 (b'Oyonnax', 14.0),
 (b'Pau', 20.0),
 (b'Stade Fran\xc3\xa7ais', 22.0),
 (b'Grenoble', 24.0),
 (b'La Rochelle', 26.0),
 (b'Castres', 29.0),
 (b'Brive', 32.0),
 (b'Montpellier', 32.0),
 (b'Bordeaux', 33.0),
 (b'Toulon', 36.0),
 (b'Toulouse', 36.0),
 (b'Clermont', 38.0),
 (b'Racing', 40.0)]

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [15]:
Toulon_score = r.zscore('city_scores','Toulon')
print(Toulon_score)

36.0


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [16]:
r.zrevrank('city_scores','Toulon')

3

**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [17]:
r.zrevrange('city_scores', 0, 2, withscores=True)

[(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]

In [18]:
min_score=35
r.zrangebyscore('city_scores',min_score+1, float('inf') , withscores=True)

[(b'Toulon', 36.0),
 (b'Toulouse', 36.0),
 (b'Clermont', 38.0),
 (b'Racing', 40.0)]

### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [19]:
r.hset("user:1","id","1")
r.hset("user:1","name","Jean")
r.hset("user:1","age","22")
r.hgetall("user:1")

{b'id': b'1', b'name': b'Jean', b'age': b'22'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [20]:
r.hset("user:1","city","Lyon")
r.hset("user:1","name","Paul")
r.hgetall("user:1")

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [21]:
import json
http_method="put"
url1 = "http://my-api.fr/user/10"
answer= {"id": 10, "name": "jean"}
cache_key_put = f"{http_method}:{url1}"
answer_json = json.dumps(answer)
r.set(cache_key_put,answer_json)
print(f"PUT request cached successfully for key: {cache_key_put}")
r.get(cache_key_put)

PUT request cached successfully for key: put:http://my-api.fr/user/10


b'{"id": 10, "name": "jean"}'

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [22]:
http_method="get"
url2 = "http://my-api.fr/user?size=2"
answer2= [{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]
cache_key_get = f"{http_method}:{url2}"
answer_json = json.dumps(answer2)
r.set(cache_key_get,answer_json)
print(f"GET request cached successfully for key: {cache_key_get}")
r.get(cache_key_get)

GET request cached successfully for key: get:http://my-api.fr/user?size=2


b'[{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]'

**Q** : Create a set of cache request keys.

In [23]:
urls=[cache_key_put,cache_key_get]
r.sadd("request keys",*urls)

2

**Q** : Check if the GET request on http: //http://my-api.fr/user

In [24]:
get=r.sismember("request keys","get:http://my-api.fr/user")
print(get)

0


**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [25]:
put=r.sismember("request keys","put:http://my-api.fr/user/10")
print(put)

1


## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [26]:
!redis-cli flushall

'redis-cli' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.
